In [ ]:
!pip install transformers
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [ ]:
!pip install -U transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# 1. Install Library

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset, Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
# Load CSV dengan pandas
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")

# Konversi ke Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Gabungkan jika ingin buat DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
def format_example(example):
    return {'text': f"{example['input']} {example['output']}"}

dataset = dataset.map(format_example)

In [ ]:
# Load model GPT-2 yang sudah di-train dengan bahasa Indonesia
tokenizer = GPT2Tokenizer.from_pretrained("cahya/gpt2-small-indonesian-522M")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("cahya/gpt2-small-indonesian-522M")
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
trainer.train()

In [ ]:
# (Optional) STEP 8: Simpan model & tokenizer
trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")

In [ ]:
model_path = "https://drive.google.com/drive/folders/1jZmzzWl_TbhhSmjDnhtwicgORaSO7xn0?usp=sharing"

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

In [ ]:
from transformers import pipeline

def generate_text(
    model,
    tokenizer,
    prompt,
    max_length=150,
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
    repetition_penalty=1.0,
    eos_token_id=None
):
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    outputs = generator(
        prompt,
        max_length=max_length,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        num_return_sequences=num_return_sequences,
        repetition_penalty=repetition_penalty,
        eos_token_id=eos_token_id,
    )

    return [out["generated_text"] for out in outputs]


In [ ]:
prompt = (
    "Tugas: Buatlah narasi cerita deskriptif berdasarkan objek berikut.\n"
    "Objek: [pohon]\n"
    "Narasi:"
)


results = generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=100,
    temperature=0.7,
    top_k=30,
    top_p=0.92,
    num_return_sequences=3,
    repetition_penalty=1.5,
    eos_token_id=tokenizer.eos_token_id
)

for i, res in enumerate(results):
    print(f"\n=== Generated Text {i+1} ===\n{res}")


In [ ]:
import shutil
shutil.make_archive('gpt2-finetuned', 'zip', './gpt2-finetuned')


In [ ]:
from google.colab import files
files.download('gpt2-finetuned.zip')
